In [1]:
import pandas as pd

In [2]:
gender = pd.read_csv("gender_submission.csv")
train = pd.read_csv("train.csv") #дата для тренировки
test = pd.read_csv("test.csv") #дата для проверки

In [3]:
train["Age"] = train["Age"].fillna(train["Age"].median())
train["Embarked"] = train["Embarked"].fillna("S")

In [8]:
data_y = train[["Survived"]]
data_x = train[["Sex", "Pclass", "Age", "SibSp", "Parch", "Fare", "Embarked"]]

test = test[["Sex","Pclass", "Age", "SibSp", "Parch", "Fare", "Embarked"]]

In [9]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False).set_output(transform='pandas')
ohetranseform = ohe.fit_transform(data_x[["Sex", "Embarked"]]) #we have to put in a dataframe with [[]]
data_x = pd.concat([data_x, ohetranseform], axis=1).drop(columns = ["Sex", "Embarked"])

In [10]:
ohetranseform = ohe.fit_transform(test[["Sex", "Embarked"]])
test = pd.concat([test, ohetranseform], axis=1).drop(columns = ["Sex", "Embarked"])

In [12]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, train_size=0.2)

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [14]:
params = {
    'criterion':  ['log_loss', 'entropy'],
    'max_depth':  [None, 2, 4, 6, 8, 10],
    'max_features': [None, 'sqrt', 'log2', 0.2, 0.4, 0.6, 0.8],
    'splitter': ['best', 'random']
}

clf = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=params,
    cv=5,
    n_jobs=5,
    verbose=1,
)


In [15]:
clf = clf.fit(train_x, train_y)

Fitting 5 folds for each of 168 candidates, totalling 840 fits


In [16]:
pred = clf.predict(test_x)
accuracy_score(test_y, pred)

0.7798036465638148

In [17]:
print (clf.best_params_)
clf.best_score_

{'criterion': 'entropy', 'max_depth': 2, 'max_features': 0.4, 'splitter': 'best'}


0.8426984126984127

In [18]:
clf_best = DecisionTreeClassifier(criterion="entropy", max_depth=2, max_features=0.4, splitter="best")

In [48]:
clf_best.fit(train_x, train_y)
predict = clf_best.predict(test)

index = []
for number in range(892, 1310):
    index.append(number)

In [64]:
df = pd.DataFrame(data = predict, index = index, columns=["Survived"])
df.index.name = "Passenger_id"
results = df.to_csv("Results.csv")